In [1]:
!pip install dynet
!git clone https://github.com/neubig/nn4nlp-code.git

    100% |████████████████████████████████| 27.6MB 1.1MB/s 
    100% |████████████████████████████████| 3.3MB 10.5MB/s 
Cloning into 'nn4nlp-code'...
remote: Counting objects: 372, done.
remote: Total 372 (delta 0), reused 0 (delta 0), pack-reused 372
Receiving objects: 100% (372/372), 6.33 MiB | 19.81 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [0]:
from __future__ import print_function
import time

start = time.time()

from collections import Counter, defaultdict
import random
import math
import sys
import argparse

import dynet as dy
import numpy as np

In [0]:
# format of files: each line is "word1 word2 ..."
train_file = "nn4nlp-code/data/ptb/train.txt"
test_file = "nn4nlp-code/data/ptb/valid.txt"

w2i = defaultdict(lambda: len(w2i))


def read(fname):
    """
    Read a file where each line is of the form "word1 word2 ..."
    Yields lists of the form [word1, word2, ...]
    """
    with open(fname, "r") as fh:
        for line in fh:
            sent = [w2i[x] for x in line.strip().split()]
            sent.append(w2i["<s>"])
            yield sent


train = list(read(train_file))
nwords = len(w2i)
test = list(read(test_file))
S = w2i["<s>"]
assert (nwords == len(w2i))

In [0]:
# DyNet Starts
model = dy.Model()
trainer = dy.AdamTrainer(model)

# Lookup parameters for word embeddings
EMBED_SIZE = 64
HIDDEN_SIZE = 128
WORDS_LOOKUP = model.add_lookup_parameters((nwords, EMBED_SIZE))

# Word-level LSTM (layers=1, input=64, output=128, model)
RNN = dy.LSTMBuilder(1, EMBED_SIZE, HIDDEN_SIZE, model)

# Softmax weights/biases on top of LSTM outputs
W_sm = model.add_parameters((nwords, HIDDEN_SIZE))
b_sm = model.add_parameters(nwords)


# Build the language model graph
def calc_lm_loss(sent):
    dy.renew_cg()
    # parameters -> expressions
    W_exp = dy.parameter(W_sm)
    b_exp = dy.parameter(b_sm)

    # initialize the RNN
    f_init = RNN.initial_state()

    # get the wids and masks for each step
    tot_words = len(sent)

    # start the rnn by inputting "<s>"
    s = f_init.add_input(WORDS_LOOKUP[S])

    # feed word vectors into the RNN and predict the next word
    losses = []
    for wid in sent:
        # calculate the softmax and loss
        score = W_exp * s.output() + b_exp
        loss = dy.pickneglogsoftmax(score, wid)
        losses.append(loss)
        # update the state of the RNN
        wemb = WORDS_LOOKUP[wid]
        s = s.add_input(wemb)

    return dy.esum(losses), tot_words


# Sort training sentences in descending order and count minibatches
train_order = range(len(train))

In [0]:
print("startup time: %r" % (time.time() - start))
# Perform training
start = time.time()
i = all_time = dev_time = all_tagged = this_words = this_loss = 0
for ITER in range(100):
    random.shuffle(train_order)
    for sid in train_order:
        i += 1
        if i % int(500) == 0:
            trainer.status()
            print(this_loss / this_words, file=sys.stderr)
            all_tagged += this_words
            this_loss = this_words = 0
            all_time = time.time() - start
        if i % int(10000) == 0:
            dev_start = time.time()
            dev_loss = dev_words = 0
            for sent in test:
                loss_exp, mb_words = calc_lm_loss(sent)
                dev_loss += loss_exp.scalar_value()
                dev_words += mb_words
            dev_time += time.time() - dev_start
            train_time = time.time() - start - dev_time
            print("nll=%.4f, ppl=%.4f, words=%r, time=%.4f, word_per_sec=%.4f" % (
            dev_loss / dev_words, math.exp(dev_loss / dev_words), dev_words, train_time, all_tagged / train_time))
        # train on the minibatch
        loss_exp, mb_words = calc_lm_loss(train[sid])
        this_loss += loss_exp.scalar_value()
        this_words += mb_words
        loss_exp.backward()
        trainer.update()
    print("epoch %r finished" % ITER)
    trainer.update_epoch(1.0)

startup time: 1.5743510723114014


7.13827299469
6.68372788932
6.56000968153
6.48601439531
6.41959502674
6.37209053545
6.2874563398
6.24974389991
6.20393947699
6.22451080823
6.03801972917
6.12050731624
6.06029874266
6.03172786914
5.98755370178
5.97345763071
5.93895772363
5.8454166822
5.86689922595
5.81222066633


nll=5.8437, ppl=345.0623, words=73760, time=541.3720, word_per_sec=409.1290


5.79615720606
5.80938518573
5.72593288831
5.77341154408
5.74417290724
5.67134200219
5.60873633325
5.71535864284
5.76652435086
5.72160954594
5.63925532553
5.66350005485
5.63274236004
5.69882071044
5.57804742479
5.58517945686
5.56072322416
5.6069517123
5.49212287856
5.66331230684

nll=5.5961, ppl=269.3728, words=73760, time=1102.3300, word_per_sec=400.6649


...

4.80687811038
4.81155921894
4.75644301998
4.74821407118


nll=5.1653, ppl=175.0879, words=73760, time=10737.0382, word_per_sec=349.8818


4.71188926205
4.73072816182
4.77958321832
4.72241238893
4.81545596829
4.78612998041
4.75512575462
4.70760169908
4.75316895817
4.76525193084
4.84469466928
4.70227915235
4.70296871699
4.75268968841
4.78268377919
4.84372124714
4.77149611795
4.76996271103
4.76947084732
4.78165042415


nll=5.1784, ppl=177.4007, words=73760, time=11298.5882, word_per_sec=352.0965


4.73218762579
4.80014401729
4.747198863
4.79126945858
4.86790455179
4.76561869318
4.83957683001
4.81246307843
4.81886903976
4.8017248505
4.7647577226
4.79305894896
4.87221781529
4.70734495608
4.83874116387
4.73478614637
4.74540598692
4.76766161521
4.76566182696
